In [1]:
import pandas as pd
#import numpy as np
import requests #api
import json #json file
#from datetime import datetime

#import matplotlib.pyplot as plt
#from scipy.stats import linregress #stastistical analysis

from pathlib import Path #reading csv - connecting to the csv merges mac and windows

from sodapy import Socrata #used for cdc API fetch

import sqlite3 as sq
from sqlalchemy.ext.automap import automap_base
from sqlalchemy import create_engine, inspect, func
from sqlalchemy.orm import Session

import geopandas as gpd

## COVID-19 Case Surveillance Public Use Data with Geography - Isabel

In [2]:
#https://data.cdc.gov/Case-Surveillance/COVID-19-Case-Surveillance-Public-Use-Data-with-Ge/n8mc-b4w4
case_surv_url = "https://data.cdc.gov/resource/n8mc-b4w4.json?"
case_surv_data = requests.get(case_surv_url).json()

case_surv_df = pd.DataFrame(case_surv_data)
#case_surv_df

In [3]:
CA_case_surv_url = "https://data.cdc.gov/resource/n8mc-b4w4.json?res_state=CA"
CA_case_surv_data = requests.get(case_surv_url).json()

CA_case_surv_df = pd.DataFrame(CA_case_surv_data)
#CA_case_surv_df

In [4]:
#CA_case_surv_df["death_yn"].unique()

In [5]:
#CA_case_surv_df["hosp_yn"].unique()

In [6]:
#CA_case_surv_df["hosp_yn"].value_counts()

In [7]:
client = Socrata("data.cdc.gov",
                  "iwYpUxUglJyjEqOoTODioBzcV",
                  username="isabelmwork@gmail.com",
                  password="APIkey2023")
CA_case_surv_query = client.get("n8mc-b4w4", res_state="CA", limit=100000)
CA_case_surv_df = pd.DataFrame.from_records(CA_case_surv_query)

In [8]:
CA_case_surv_df.head()

,case_month,res_state,state_fips_code,res_county,county_fips_code,age_group,sex,race,ethnicity,process,exposure_yn,current_status,symptom_status,hosp_yn,icu_yn,death_yn,case_onset_interval,underlying_conditions_yn,case_positive_specimen
0,2021-01,CA,06,SAN BERNARDINO,06071,18 to 49 years,Male,White,Non-Hispanic/Latino,Missing,Missing,Laboratory-confirmed case,Unknown,Missing,Missing,Missing,NaN,NaN,NaN
1,2022-02,CA,06,SOLANO,06095,18 to 49 years,Male,White,Non-Hispanic/Latino,Missing,Missing,Laboratory-confirmed case,Unknown,Missing,Missing,Missing,NaN,NaN,NaN
2,2021-11,CA,06,YUBA,06115,18 to 49 years,Male,White,Non-Hispanic/Latino,Missing,Missing,Laboratory-confirmed case,Unknown,Missing,Missing,Missing,NaN,NaN,NaN
3,2020-07,CA,06,BUTTE,06007,50 to 64 years,Male,White,Non-Hispanic/Latino,Missing,Missing,Laboratory-confirmed case,Symptomatic,No,Missing,NA,0.0,NaN,NaN
4,2022-01,CA,06,CALAVERAS,06009,50 to 64 years,Male,White,Non-Hispanic/Latino,Missing,Missing,Laboratory-confirmed case,Unknown,Missing,Missing,Missing,NaN,NaN,NaN


In [9]:
CA_case_surv_df = CA_case_surv_df.sort_values(by=["res_county"])
CA_case_surv_df.head()

,case_month,res_state,state_fips_code,res_county,county_fips_code,age_group,sex,race,ethnicity,process,exposure_yn,current_status,symptom_status,hosp_yn,icu_yn,death_yn,case_onset_interval,underlying_conditions_yn,case_positive_specimen
16247,2022-05,CA,06,ALAMEDA,06001,0 - 17 years,Unknown,White,Non-Hispanic/Latino,Missing,Missing,Laboratory-confirmed case,Unknown,Missing,Missing,Missing,NaN,NaN,NaN
16748,2022-05,CA,06,ALAMEDA,06001,0 - 17 years,Unknown,White,Non-Hispanic/Latino,Missing,Missing,Laboratory-confirmed case,Unknown,Missing,Missing,Missing,NaN,NaN,NaN
96223,2022-03,CA,06,ALAMEDA,06001,18 to 49 years,Male,White,Unknown,Missing,Missing,Laboratory-confirmed case,Asymptomatic,Missing,Missing,Missing,NaN,NaN,NaN
74163,2022-03,CA,06,ALAMEDA,06001,18 to 49 years,Male,Unknown,Unknown,Missing,Missing,Laboratory-confirmed case,Unknown,Missing,Missing,Missing,NaN,NaN,NaN
64507,2022-05,CA,06,ALAMEDA,06001,65+ years,Female,Unknown,Unknown,Missing,Missing,Laboratory-confirmed case,Unknown,Missing,Missing,Missing,NaN,NaN,NaN


In [10]:
#CA_case_surv_df["res_county"] = CA_case_surv_df["res_county"].str.lower().str.capitalize()

In [11]:
CA_case_surv_df.head()

,case_month,res_state,state_fips_code,res_county,county_fips_code,age_group,sex,race,ethnicity,process,exposure_yn,current_status,symptom_status,hosp_yn,icu_yn,death_yn,case_onset_interval,underlying_conditions_yn,case_positive_specimen
16247,2022-05,CA,06,ALAMEDA,06001,0 - 17 years,Unknown,White,Non-Hispanic/Latino,Missing,Missing,Laboratory-confirmed case,Unknown,Missing,Missing,Missing,NaN,NaN,NaN
16748,2022-05,CA,06,ALAMEDA,06001,0 - 17 years,Unknown,White,Non-Hispanic/Latino,Missing,Missing,Laboratory-confirmed case,Unknown,Missing,Missing,Missing,NaN,NaN,NaN
96223,2022-03,CA,06,ALAMEDA,06001,18 to 49 years,Male,White,Unknown,Missing,Missing,Laboratory-confirmed case,Asymptomatic,Missing,Missing,Missing,NaN,NaN,NaN
74163,2022-03,CA,06,ALAMEDA,06001,18 to 49 years,Male,Unknown,Unknown,Missing,Missing,Laboratory-confirmed case,Unknown,Missing,Missing,Missing,NaN,NaN,NaN
64507,2022-05,CA,06,ALAMEDA,06001,65+ years,Female,Unknown,Unknown,Missing,Missing,Laboratory-confirmed case,Unknown,Missing,Missing,Missing,NaN,NaN,NaN


In [12]:
CA_case_surv_df[["year", "month"]] = CA_case_surv_df["case_month"].str.split("-", expand = True)

In [13]:
CA_case_surv_df.head()

,case_month,res_state,state_fips_code,res_county,county_fips_code,age_group,sex,race,ethnicity,process,...,current_status,symptom_status,hosp_yn,icu_yn,death_yn,case_onset_interval,underlying_conditions_yn,case_positive_specimen,year,month
16247,2022-05,CA,06,ALAMEDA,06001,0 - 17 years,Unknown,White,Non-Hispanic/Latino,Missing,...,Laboratory-confirmed case,Unknown,Missing,Missing,Missing,NaN,NaN,NaN,2022,05
16748,2022-05,CA,06,ALAMEDA,06001,0 - 17 years,Unknown,White,Non-Hispanic/Latino,Missing,...,Laboratory-confirmed case,Unknown,Missing,Missing,Missing,NaN,NaN,NaN,2022,05
96223,2022-03,CA,06,ALAMEDA,06001,18 to 49 years,Male,White,Unknown,Missing,...,Laboratory-confirmed case,Asymptomatic,Missing,Missing,Missing,NaN,NaN,NaN,2022,03
74163,2022-03,CA,06,ALAMEDA,06001,18 to 49 years,Male,Unknown,Unknown,Missing,...,Laboratory-confirmed case,Unknown,Missing,Missing,Missing,NaN,NaN,NaN,2022,03
64507,2022-05,CA,06,ALAMEDA,06001,65+ years,Female,Unknown,Unknown,Missing,...,Laboratory-confirmed case,Unknown,Missing,Missing,Missing,NaN,NaN,NaN,2022,05


In [14]:
CA_case_sel_df = CA_case_surv_df[["res_county", "year", "month", "age_group", \
                                  "sex", "race", "ethnicity", "hosp_yn", "icu_yn", "death_yn", \
                                  "underlying_conditions_yn"]]

In [15]:
CA_case_sel_df.head()

,res_county,year,month,age_group,sex,race,ethnicity,hosp_yn,icu_yn,death_yn,underlying_conditions_yn
16247,ALAMEDA,2022,05,0 - 17 years,Unknown,White,Non-Hispanic/Latino,Missing,Missing,Missing,NaN
16748,ALAMEDA,2022,05,0 - 17 years,Unknown,White,Non-Hispanic/Latino,Missing,Missing,Missing,NaN
96223,ALAMEDA,2022,03,18 to 49 years,Male,White,Unknown,Missing,Missing,Missing,NaN
74163,ALAMEDA,2022,03,18 to 49 years,Male,Unknown,Unknown,Missing,Missing,Missing,NaN
64507,ALAMEDA,2022,05,65+ years,Female,Unknown,Unknown,Missing,Missing,Missing,NaN


In [16]:
CA_case_sel_df = CA_case_sel_df.rename(columns = {"res_county":"county"})

In [17]:
CA_case_sel_df = CA_case_sel_df.sort_values(by= \
                                            ["county", "year", "month"], \
                                            ascending= \
                                            [True, True, True])

In [18]:
CA_case_sel_df = CA_case_sel_df.reset_index(drop = True)
CA_case_sel_df["id"] = CA_case_sel_df.index

In [19]:
CA_case_sel_df = CA_case_sel_df[['id', 'county', 'year', 'month', 'age_group', 'sex', 'race', 'ethnicity',
       'hosp_yn', 'icu_yn', 'death_yn', 'underlying_conditions_yn']]

In [20]:
CA_case_sel_df["hosp_yn"].unique()

array(['No', 'Missing', 'Yes', 'Unknown'], dtype=object)

In [21]:
CA_case_sel_df["hosp_yn"] = CA_case_sel_df["hosp_yn"].mask(CA_case_sel_df["hosp_yn"].str.contains('Missing'), "Unknown")


In [22]:
CA_case_sel_df["hosp_yn"].unique()

array(['No', 'Unknown', 'Yes'], dtype=object)

In [23]:
CA_case_sel_df["death_yn"].unique()

array(['Missing', 'NA', 'Yes'], dtype=object)

In [24]:
CA_case_sel_df["death_yn"] = CA_case_sel_df["death_yn"].mask(CA_case_sel_df["death_yn"].str.contains('Missing'), "Unknown")


In [25]:
CA_case_sel_df["death_yn"].unique()

array(['Unknown', 'NA', 'Yes'], dtype=object)

In [26]:
num_months = list(CA_case_sel_df["month"].unique())
num_months.sort()
#sort_num_months = num_months.sort()
#print(sort_num_months)

In [27]:
word_months = ["January", "February", "March", "April", "May", "June", "July", "August", "September", "October", "November", "December"]


In [28]:
count = 0
for month in num_months:
    
    CA_case_sel_df["month"] = CA_case_sel_df["month"].mask(CA_case_sel_df["month"].str.contains(month), word_months[count])
    count += 1
    

In [29]:
CA_case_sel_df["month"].unique()

array(['August', 'November', 'December', 'January', 'February', 'July',
       'September', 'October', 'March', 'April', 'May', 'June'],
      dtype=object)

In [30]:
CA_case_sel_df.head()

,id,county,year,month,age_group,sex,race,ethnicity,hosp_yn,icu_yn,death_yn,underlying_conditions_yn
0,0,ALAMEDA,2020,August,18 to 49 years,Male,NA,NA,No,Missing,Unknown,NaN
1,1,ALAMEDA,2020,August,65+ years,Female,Unknown,Unknown,Unknown,Missing,Unknown,NaN
2,2,ALAMEDA,2020,August,18 to 49 years,Male,NA,NA,No,Missing,Unknown,NaN
3,3,ALAMEDA,2020,August,18 to 49 years,Male,NA,NA,Unknown,Missing,Unknown,NaN
4,4,ALAMEDA,2020,August,18 to 49 years,Male,NA,NA,Unknown,Missing,Unknown,NaN


In [31]:
CA_case_sel_df.tail()

,id,county,year,month,age_group,sex,race,ethnicity,hosp_yn,icu_yn,death_yn,underlying_conditions_yn
99995,99995,YUBA,2023,August,18 to 49 years,Female,White,Non-Hispanic/Latino,Unknown,Missing,Unknown,NaN
99996,99996,YUBA,2023,August,65+ years,Female,NA,NA,Unknown,Missing,Unknown,NaN
99997,99997,YUBA,2023,August,18 to 49 years,Female,White,Non-Hispanic/Latino,Unknown,Missing,Unknown,NaN
99998,99998,YUBA,2023,August,18 to 49 years,Female,White,Non-Hispanic/Latino,Unknown,Missing,Unknown,NaN
99999,99999,YUBA,2023,August,65+ years,Female,NA,NA,Unknown,Missing,Unknown,NaN


In [32]:
CA_case_sel_df.columns.values

array(['id', 'county', 'year', 'month', 'age_group', 'sex', 'race',
       'ethnicity', 'hosp_yn', 'icu_yn', 'death_yn',
       'underlying_conditions_yn'], dtype=object)

In [33]:
CA_case_sel_df["year"] = pd.to_numeric(CA_case_sel_df["year"])

In [34]:
#CA_case_sel_df["month"] = pd.to_numeric(CA_case_sel_df["month"])

In [35]:
CA_case_sel_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 12 columns):
 #   Column                    Non-Null Count   Dtype 
---  ------                    --------------   ----- 
 0   id                        100000 non-null  int64 
 1   county                    100000 non-null  object
 2   year                      100000 non-null  int64 
 3   month                     100000 non-null  object
 4   age_group                 100000 non-null  object
 5   sex                       100000 non-null  object
 6   race                      100000 non-null  object
 7   ethnicity                 100000 non-null  object
 8   hosp_yn                   100000 non-null  object
 9   icu_yn                    100000 non-null  object
 10  death_yn                  100000 non-null  object
 11  underlying_conditions_yn  437 non-null     object
dtypes: int64(2), object(10)
memory usage: 9.2+ MB


In [36]:
hosp_yes = CA_case_sel_df[CA_case_sel_df["hosp_yn"] == "Unknown"]

In [37]:
hosp_yes_county = hosp_yes[hosp_yes["county"] == "KERN"]

In [38]:
hosp_yes_county["county"].unique()

array(['KERN'], dtype=object)

In [39]:
#hosp_yes_county

In [40]:
by_year_month = hosp_yes_county.groupby(["year", "month"])

In [41]:
by_year_month.count()

id  county  age_group  sex  race  ethnicity  hosp_yn  icu_yn  \
year month                                                                     
2020 August      9       9          9    9     9          9        9       9   
     December  327     327        327  327   327        327      327     327   
     June       14      14         14   14    14         14       14      14   
2021 October    12      12         12   12    12         12       12      12   
2022 August    125     125        125  125   125        125      125     125   
     February  322     322        322  322   322        322      322     322   
     January    27      27         27   27    27         27       27      27   
     July      993     993        993  993   993        993      993     993   
2023 July       21      21         21   21    21         21       21      21   

               death_yn  underlying_conditions_yn  
year month                                         
2020 August           9                         0  
     December       327                         3  
     June            14                         0  
2021 October         12                         0  
2022 August         125                         0  
     February       322                         3  
     January         27                         0  
     July           993                         5  
2023 July            21                         0

In [42]:
CA_case_sel_df.to_csv('../Data/CA_case_surv.csv', index = False)

In [43]:
CA_case_sel_df.to_json('../Data/CA_case_surv.json')

## Statewide COVID-19 Vaccines Administered By County - Nicole

In [44]:
#https://data.chhs.ca.gov/dataset/vaccine-progress-dashboard
CA_vax_path = Path("../Data/covid19vaccinesbycounty.csv")
CA_vax_df = pd.read_csv(CA_vax_path)
CA_vax_df.head()

,county,administered_date,total_doses,cumulative_total_doses,partially_vaccinated,total_partially_vaccinated,fully_vaccinated,cumulative_fully_vaccinated,at_least_one_dose,cumulative_at_least_one_dose,california_flag,up_to_date_count,cumulative_up_to_date_count
0,Alameda,2020-01-05,0,0,0,0,0,0,0,0,NaN,0,0
1,Alameda,2020-07-27,0,0,0,0,0,0,0,0,NaN,0,0
2,Alameda,2020-07-29,0,1,0,1,0,0,0,1,NaN,0,0
3,Alameda,2020-07-30,0,1,0,1,0,0,0,1,NaN,0,0
4,Alameda,2020-07-31,0,1,0,1,0,0,0,1,NaN,0,0


In [45]:
#county, admin date - month and year - group by, find max cumulative of take all of cumulative columns
cleaned_vaccine = CA_vax_df[['county','administered_date','cumulative_total_doses','cumulative_fully_vaccinated',\
                             'cumulative_at_least_one_dose','cumulative_up_to_date_count']]
cleaned_vaccine.head()

,county,administered_date,cumulative_total_doses,cumulative_fully_vaccinated,cumulative_at_least_one_dose,cumulative_up_to_date_count
0,Alameda,2020-01-05,0,0,0,0
1,Alameda,2020-07-27,0,0,0,0
2,Alameda,2020-07-29,1,0,1,0
3,Alameda,2020-07-30,1,0,1,0
4,Alameda,2020-07-31,1,0,1,0


In [46]:
# cleaned_vaccine['year'] = cleaned_vaccine['administered_date'].dt.year
# cleaned_vaccine['month'] = cleaned_vaccine['administered_date'].dt.month
# cleaned_vaccine.head()
#defaults turning datetme to timestamp because more secure

In [47]:
cleaned_vaccine.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 71796 entries, 0 to 71795
Data columns (total 6 columns):
 #   Column                        Non-Null Count  Dtype 
---  ------                        --------------  ----- 
 0   county                        71796 non-null  object
 1   administered_date             71796 non-null  object
 2   cumulative_total_doses        71796 non-null  int64 
 3   cumulative_fully_vaccinated   71796 non-null  int64 
 4   cumulative_at_least_one_dose  71796 non-null  int64 
 5   cumulative_up_to_date_count   71796 non-null  int64 
dtypes: int64(4), object(2)
memory usage: 3.3+ MB


In [48]:
# cleaned_vaccine['administered_date'] = pd.to_datetime(cleaned_vaccine['administered_date'])
# cleaned_vaccine.info()
#not going to change into date time because i will  be using the breaking a string as their own with '-'

In [49]:
#line 14 and 15 CA case urv to break up the string so dates are 01-05-2020 to year and month as 2020 and 07
# Get rid of changing into datetime
cleaned_vaccine[['year', 'month', 'day']] = \
    cleaned_vaccine['administered_date'].str.split('-', n=2, expand = True)
#delimiter is for column info not title
#n: (int type_ number of splits, default is -1)
cleaned_vaccine.head()

/var/folders/ds/jk39g9xd62v_5kxdlxcxn0800000gn/T/ipykernel_6966/3244646067.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cleaned_vaccine[['year', 'month', 'day']] = \
/var/folders/ds/jk39g9xd62v_5kxdlxcxn0800000gn/T/ipykernel_6966/3244646067.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cleaned_vaccine[['year', 'month', 'day']] = \
/var/folders/ds/jk39g9xd62v_5kxdlxcxn0800000gn/T/ipykernel_6966/3244646067.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a Data

,county,administered_date,cumulative_total_doses,cumulative_fully_vaccinated,cumulative_at_least_one_dose,cumulative_up_to_date_count,year,month,day
0,Alameda,2020-01-05,0,0,0,0,2020,01,05
1,Alameda,2020-07-27,0,0,0,0,2020,07,27
2,Alameda,2020-07-29,1,0,1,0,2020,07,29
3,Alameda,2020-07-30,1,0,1,0,2020,07,30
4,Alameda,2020-07-31,1,0,1,0,2020,07,31


In [50]:
#deleting extraneous columns
del cleaned_vaccine['administered_date']
cleaned_vaccine.head()

,county,cumulative_total_doses,cumulative_fully_vaccinated,cumulative_at_least_one_dose,cumulative_up_to_date_count,year,month,day
0,Alameda,0,0,0,0,2020,01,05
1,Alameda,0,0,0,0,2020,07,27
2,Alameda,1,0,1,0,2020,07,29
3,Alameda,1,0,1,0,2020,07,30
4,Alameda,1,0,1,0,2020,07,31


In [51]:
#deleting extraneous columns
del cleaned_vaccine['day']
cleaned_vaccine.head()

,county,cumulative_total_doses,cumulative_fully_vaccinated,cumulative_at_least_one_dose,cumulative_up_to_date_count,year,month
0,Alameda,0,0,0,0,2020,01
1,Alameda,0,0,0,0,2020,07
2,Alameda,1,0,1,0,2020,07
3,Alameda,1,0,1,0,2020,07
4,Alameda,1,0,1,0,2020,07


In [52]:
#we want to look at data for each month in each county
cleaned_vaccine.groupby(['month','year'])
cleaned_vaccine.head()

,county,cumulative_total_doses,cumulative_fully_vaccinated,cumulative_at_least_one_dose,cumulative_up_to_date_count,year,month
0,Alameda,0,0,0,0,2020,01
1,Alameda,0,0,0,0,2020,07
2,Alameda,1,0,1,0,2020,07
3,Alameda,1,0,1,0,2020,07
4,Alameda,1,0,1,0,2020,07


In [53]:
#finding the max in each county per month
max_month = cleaned_vaccine.groupby(['county','year','month']).max()
max_month.head()

cumulative_total_doses  cumulative_fully_vaccinated  \
county  year month                                                        
Alameda 2020 01                          0                            0   
             07                          1                            0   
             08                         43                            3   
             09                        104                           45   
             10                        121                           51   

                    cumulative_at_least_one_dose  cumulative_up_to_date_count  
county  year month                                                             
Alameda 2020 01                                0                            0  
             07                                1                            0  
             08                               40                            0  
             09                               59                            0  
             10                               72                            0

In [54]:
#troubleshooting why I am unable to search for just alameda county
print(max_month.columns)

Index(['cumulative_total_doses', 'cumulative_fully_vaccinated',
       'cumulative_at_least_one_dose', 'cumulative_up_to_date_count'],
      dtype='object')


In [55]:
#because county column is not here, after doing the groupby used the county column as the grouping keys so I cannot treat it
# as a regular column, so in the next column i reset the index

In [56]:
#just looking at alameda data to make sure cleaned correctly
max_month_reset = max_month.reset_index()
alameda_county = max_month_reset[max_month_reset['county']=='Alameda']
alameda_county.head()
#now we can see there is from Jan 2020 to Sept 2023

,county,year,month,cumulative_total_doses,cumulative_fully_vaccinated,cumulative_at_least_one_dose,cumulative_up_to_date_count
0,Alameda,2020,01,0,0,0,0
1,Alameda,2020,07,1,0,1,0
2,Alameda,2020,08,43,3,40,0
3,Alameda,2020,09,104,45,59,0
4,Alameda,2020,10,121,51,72,0


In [57]:
#renaming df
clean_complete_vaccine = max_month_reset
clean_complete_vaccine.head()

,county,year,month,cumulative_total_doses,cumulative_fully_vaccinated,cumulative_at_least_one_dose,cumulative_up_to_date_count
0,Alameda,2020,01,0,0,0,0
1,Alameda,2020,07,1,0,1,0
2,Alameda,2020,08,43,3,40,0
3,Alameda,2020,09,104,45,59,0
4,Alameda,2020,10,121,51,72,0


In [58]:
clean_complete_vaccine["year"] = pd.to_numeric(clean_complete_vaccine["year"])

In [59]:
clean_complete_vaccine["month"] = pd.to_numeric(clean_complete_vaccine["month"])

In [60]:
clean_complete_vaccine["county"] = clean_complete_vaccine["county"].str.upper()

In [61]:
clean_complete_vaccine["id"] = clean_complete_vaccine.index
clean_complete_vaccine.columns.values

array(['county', 'year', 'month', 'cumulative_total_doses',
       'cumulative_fully_vaccinated', 'cumulative_at_least_one_dose',
       'cumulative_up_to_date_count', 'id'], dtype=object)

In [62]:
clean_complete_vaccine.head()

,county,year,month,cumulative_total_doses,cumulative_fully_vaccinated,cumulative_at_least_one_dose,cumulative_up_to_date_count,id
0,ALAMEDA,2020,1,0,0,0,0,0
1,ALAMEDA,2020,7,1,0,1,0,1
2,ALAMEDA,2020,8,43,3,40,0,2
3,ALAMEDA,2020,9,104,45,59,0,3
4,ALAMEDA,2020,10,121,51,72,0,4


In [63]:
clean_complete_vaccine.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2480 entries, 0 to 2479
Data columns (total 8 columns):
 #   Column                        Non-Null Count  Dtype 
---  ------                        --------------  ----- 
 0   county                        2480 non-null   object
 1   year                          2480 non-null   int64 
 2   month                         2480 non-null   int64 
 3   cumulative_total_doses        2480 non-null   int64 
 4   cumulative_fully_vaccinated   2480 non-null   int64 
 5   cumulative_at_least_one_dose  2480 non-null   int64 
 6   cumulative_up_to_date_count   2480 non-null   int64 
 7   id                            2480 non-null   int64 
dtypes: int64(7), object(1)
memory usage: 155.1+ KB


In [64]:
clean_complete_vaccine['Date'] = clean_complete_vaccine['year'].astype(str) + '-' + cleaned_vaccine['month'].astype(str)
clean_complete_vaccine

,county,year,month,cumulative_total_doses,cumulative_fully_vaccinated,cumulative_at_least_one_dose,cumulative_up_to_date_count,id,Date
0,ALAMEDA,2020,1,0,0,0,0,0,2020-01
1,ALAMEDA,2020,7,1,0,1,0,1,2020-07
2,ALAMEDA,2020,8,43,3,40,0,2,2020-07
3,ALAMEDA,2020,9,104,45,59,0,3,2020-07
4,ALAMEDA,2020,10,121,51,72,0,4,2020-07
...,...,...,...,...,...,...,...,...,...
2475,YUBA,2023,5,109747,40271,44406,7025,2475,2023-10
2476,YUBA,2023,6,109892,40278,44426,7154,2476,2023-10
2477,YUBA,2023,7,109983,40279,44448,7239,2477,2023-10
2478,YUBA,2023,8,110114,40280,44472,7364,2478,2023-10


In [65]:
clean_complete_vaccine.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2480 entries, 0 to 2479
Data columns (total 9 columns):
 #   Column                        Non-Null Count  Dtype 
---  ------                        --------------  ----- 
 0   county                        2480 non-null   object
 1   year                          2480 non-null   int64 
 2   month                         2480 non-null   int64 
 3   cumulative_total_doses        2480 non-null   int64 
 4   cumulative_fully_vaccinated   2480 non-null   int64 
 5   cumulative_at_least_one_dose  2480 non-null   int64 
 6   cumulative_up_to_date_count   2480 non-null   int64 
 7   id                            2480 non-null   int64 
 8   Date                          2480 non-null   object
dtypes: int64(7), object(2)
memory usage: 174.5+ KB


In [66]:
clean_complete_vaccine['Date'] = clean_complete_vaccine['Date'].astype('|S')

In [67]:
clean_complete_vaccine.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2480 entries, 0 to 2479
Data columns (total 9 columns):
 #   Column                        Non-Null Count  Dtype 
---  ------                        --------------  ----- 
 0   county                        2480 non-null   object
 1   year                          2480 non-null   int64 
 2   month                         2480 non-null   int64 
 3   cumulative_total_doses        2480 non-null   int64 
 4   cumulative_fully_vaccinated   2480 non-null   int64 
 5   cumulative_at_least_one_dose  2480 non-null   int64 
 6   cumulative_up_to_date_count   2480 non-null   int64 
 7   id                            2480 non-null   int64 
 8   Date                          2480 non-null   |S7   
dtypes: bytes56(1), int64(7), object(1)
memory usage: 172.1+ KB


In [68]:
clean_complete_vaccine.to_csv('../Data/VaccineComplete.csv', index = False)
#index = False to exclude row number in output

In [69]:
clean_complete_vaccine.to_json('../Data/VaccineComplete.json')

## Vaccination Zipcodes - Adriana

In [70]:
df = pd.read_csv("COVID19VaccineRecords.csv", low_memory=False, encoding='utf-8')

In [71]:
df.head()

,_id,as_of_date,zip_code_tabulation_area,local_health_jurisdiction,county,vaccine_equity_metric_quartile,vem_source,age12_plus_population,age5_plus_population,tot_population,persons_fully_vaccinated,persons_partially_vaccinated,percent_of_population_fully_vaccinated,percent_of_population_partially_vaccinated,percent_of_population_with_1_plus_dose,up_to_date_count,redacted
0,1,2021-01-12,93618,Tulare,Tulare,1.0,Healthy Places Index Score,24482.3,28588,31470.0,51.0,252.0,0.001621,0.008008,0.009629,0,Information redacted in accordance with CA sta...
1,2,2021-01-12,95437,Mendocino,Mendocino,2.0,Healthy Places Index Score,12595.5,13932,14859.0,66.0,317.0,0.004442,0.021334,0.025776,0,Information redacted in accordance with CA sta...
2,3,2021-01-12,95991,Sutter,Sutter,1.0,Healthy Places Index Score,33300.8,37870,40861.0,114.0,835.0,0.002790,0.020435,0.023225,0,Information redacted in accordance with CA sta...
3,4,2021-01-12,93444,San Luis Obispo,San Luis Obispo,3.0,Healthy Places Index Score,18951.8,20522,21331.0,155.0,441.0,0.007266,0.020674,0.027940,0,Information redacted in accordance with CA sta...
4,5,2021-01-12,95039,Monterey,Monterey,1.0,CDPH-Derived ZCTA Score,860.0,1032,1074.0,NaN,NaN,NaN,NaN,NaN,0,Information redacted in accordance with CA sta...


In [72]:
# Define a dictionary to map the old column names to new names
renamed_columns = {
    '_id': 'ID',
    'as_of_date': 'Date',
    'zip_code_tabulation_area': 'Zip Code',
    'local_health_jurisdiction': 'Health Jurisdiction',
    'county': 'County Name',
    'vaccine_equity_metric_quartile': 'Vaccine Quartile',
    'vem_source': 'Vaccine Source',
    'age12_plus_population': 'Age 12+ Population',
    'age5_plus_population': 'Age 5+ Population',
    'tot_population': 'Total Population',
    'persons_fully_vaccinated': 'Fully Vaccinated',
    'persons_partially_vaccinated': 'Partially Vaccinated',
    'percent_of_population_fully_vaccinated': 'Percent Fully Vaccinated',
    'percent_of_population_partially_vaccinated': 'Percent Partially Vaccinated',
    'percent_of_population_with_1_plus_dose': 'Percent With 1+ Dose',
    'up_to_date_count': 'Up To Date Count',
    'redacted': 'Redacted Info'
}

# Use the rename method to apply the column name mapping
df.rename(columns=renamed_columns, inplace=True)

# Display the DataFrame with renamed columns
df.head()

,ID,Date,Zip Code,Health Jurisdiction,County Name,Vaccine Quartile,Vaccine Source,Age 12+ Population,Age 5+ Population,Total Population,Fully Vaccinated,Partially Vaccinated,Percent Fully Vaccinated,Percent Partially Vaccinated,Percent With 1+ Dose,Up To Date Count,Redacted Info
0,1,2021-01-12,93618,Tulare,Tulare,1.0,Healthy Places Index Score,24482.3,28588,31470.0,51.0,252.0,0.001621,0.008008,0.009629,0,Information redacted in accordance with CA sta...
1,2,2021-01-12,95437,Mendocino,Mendocino,2.0,Healthy Places Index Score,12595.5,13932,14859.0,66.0,317.0,0.004442,0.021334,0.025776,0,Information redacted in accordance with CA sta...
2,3,2021-01-12,95991,Sutter,Sutter,1.0,Healthy Places Index Score,33300.8,37870,40861.0,114.0,835.0,0.002790,0.020435,0.023225,0,Information redacted in accordance with CA sta...
3,4,2021-01-12,93444,San Luis Obispo,San Luis Obispo,3.0,Healthy Places Index Score,18951.8,20522,21331.0,155.0,441.0,0.007266,0.020674,0.027940,0,Information redacted in accordance with CA sta...
4,5,2021-01-12,95039,Monterey,Monterey,1.0,CDPH-Derived ZCTA Score,860.0,1032,1074.0,NaN,NaN,NaN,NaN,NaN,0,Information redacted in accordance with CA sta...


In [73]:
# Delete the last column
df = df.drop(df.columns[-1], axis=1)

In [74]:
df.head()

,ID,Date,Zip Code,Health Jurisdiction,County Name,Vaccine Quartile,Vaccine Source,Age 12+ Population,Age 5+ Population,Total Population,Fully Vaccinated,Partially Vaccinated,Percent Fully Vaccinated,Percent Partially Vaccinated,Percent With 1+ Dose,Up To Date Count
0,1,2021-01-12,93618,Tulare,Tulare,1.0,Healthy Places Index Score,24482.3,28588,31470.0,51.0,252.0,0.001621,0.008008,0.009629,0
1,2,2021-01-12,95437,Mendocino,Mendocino,2.0,Healthy Places Index Score,12595.5,13932,14859.0,66.0,317.0,0.004442,0.021334,0.025776,0
2,3,2021-01-12,95991,Sutter,Sutter,1.0,Healthy Places Index Score,33300.8,37870,40861.0,114.0,835.0,0.002790,0.020435,0.023225,0
3,4,2021-01-12,93444,San Luis Obispo,San Luis Obispo,3.0,Healthy Places Index Score,18951.8,20522,21331.0,155.0,441.0,0.007266,0.020674,0.027940,0
4,5,2021-01-12,95039,Monterey,Monterey,1.0,CDPH-Derived ZCTA Score,860.0,1032,1074.0,NaN,NaN,NaN,NaN,NaN,0


In [75]:
drops_vaccine = [
    "Age 12+ Population",
    "Age 5+ Population",
    "Vaccine Source",
    "Health Jurisdiction",
    "Up To Date Count",
    "Fully Vaccinated",
    "Partially Vaccinated",
    "Percent With 1+ Dose",
    "Vaccine Quartile",
    "ID"
    
]

df.drop(columns=drops_vaccine, inplace=True)

In [76]:
# Keeping the latest dates for each Zip Code
df.sort_values(by='Date', ascending=False, inplace=True)
df.drop_duplicates(subset='County Name', keep='first', inplace=True)

In [77]:
df

,Date,Zip Code,County Name,Total Population,Percent Fully Vaccinated,Percent Partially Vaccinated
250487,2023-09-26,93664,Fresno,380.0,0.794737,0.097368
249315,2023-09-26,95460,Mendocino,2508.0,0.868022,0.072967
249304,2023-09-26,95451,Lake,11603.0,0.593640,0.064811
249305,2023-09-26,92832,Orange,25837.0,0.719743,0.075783
249306,2023-09-26,95148,Santa Clara,48273.0,0.875810,0.056968
249307,2023-09-26,96049,Shasta,90.0,1.000000,0.877778
249308,2023-09-26,95686,San Joaquin,1262.0,0.544374,0.062599
249309,2023-09-26,95317,Merced,1203.0,0.343308,0.064838
249310,2023-09-26,94115,San Francisco,34604.0,0.858196,0.099468
249311,2023-09-26,92555,Riverside,44452.0,0.624404,0.059705


In [78]:
#HPI File
url = "https://api.healthyplacesindex.org/api/hpi?geography=zips&year=2022&indicator=hpi2score&format=json&key=721a0a48-97f3-4a3f-a794-2bc855972617"
data = requests.get(url).json()
hpi_df = pd.DataFrame(data)
hpi_df

,geoid,name,population,value,percentile,numerator,denominator,se
0,94601,94601,53039,-0.380256,0.246002,None,None,None
1,94501,94501,63821,0.584656,0.860625,None,None,None
2,94560,94560,47171,0.389073,0.740289,None,None,None
3,94587,94587,74722,0.448256,0.783701,None,None,None
4,94580,94580,30488,0.246554,0.654227,None,None,None
...,...,...,...,...,...,...,...,...
1308,93274,93274,74000,-0.566216,0.150038,None,None,None
1309,95370,95370,28096,0.040532,0.525514,None,None,None
1310,95372,95372,2056,-0.000992,0.501904,None,None,None
1311,95627,95627,3802,-0.119862,0.418126,None,None,None


In [79]:
column_name_mapping = {
    'geoid': 'Zip Code',
    'name': 'Location Name',
    'population': 'Population',
    'value': 'HPI Value',
    'percentile': 'HPI Percentile',
    'numerator': 'Numerator',
    'denominator': 'Denominator'
}

# Rename the columns using the dictionary
hpi_df.rename(columns=column_name_mapping, inplace=True)

In [80]:
# Change to float
hpi_df['HPI Value'] = hpi_df['HPI Value'].astype(float)
hpi_df

,Zip Code,Location Name,Population,HPI Value,HPI Percentile,Numerator,Denominator,se
0,94601,94601,53039,-0.380256,0.246002,None,None,None
1,94501,94501,63821,0.584656,0.860625,None,None,None
2,94560,94560,47171,0.389073,0.740289,None,None,None
3,94587,94587,74722,0.448256,0.783701,None,None,None
4,94580,94580,30488,0.246554,0.654227,None,None,None
...,...,...,...,...,...,...,...,...
1308,93274,93274,74000,-0.566216,0.150038,None,None,None
1309,95370,95370,28096,0.040532,0.525514,None,None,None
1310,95372,95372,2056,-0.000992,0.501904,None,None,None
1311,95627,95627,3802,-0.119862,0.418126,None,None,None


In [81]:
#employed File
url = "https://api.healthyplacesindex.org/api/hpi?geography=zips&year=2015&indicator=employed&format=json&key=721a0a48-97f3-4a3f-a794-2bc855972617"
data = requests.get(url).json()
employed_df = pd.DataFrame(data)
employed_df

,geoid,name,population,value,percentile,numerator,denominator,se
0,92122,92122,48071,0.671,0.275495,23390,34859,0.016413
1,91384,91384,29212,0.552,0.058980,10803,19570,0.010942
2,96052,96052,1574,0.513,0.037291,467,911,0.063222
3,91354,91354,32544,0.788,0.872146,15156,19234,0.010334
4,92026,92026,50321,0.765,0.744673,22870,29895,0.012158
...,...,...,...,...,...,...,...,...
1309,95448,95448,17407,0.794,0.893075,7988,10061,0.017021
1310,95901,95901,33455,0.632,0.166286,12571,19891,0.018237
1311,96088,96088,4690,0.577,0.085616,1466,2540,0.037082
1312,96118,96118,1779,0.663,0.248858,623,940,0.052888


In [82]:
employed_columns= {
    'geoid': 'Zip Code',
    'name': 'Location Name',
    'population': 'Employed Population',
    'value': 'Employed Value',
    'percentile': 'Employed Percentile',
    'numerator': 'Employed Numerator',
    'denominator': 'Employed Denominator'
}

# Rename the columns using the dictionary
employed_df.rename(columns=employed_columns, inplace=True)

employed_df

,Zip Code,Location Name,Employed Population,Employed Value,Employed Percentile,Employed Numerator,Employed Denominator,se
0,92122,92122,48071,0.671,0.275495,23390,34859,0.016413
1,91384,91384,29212,0.552,0.058980,10803,19570,0.010942
2,96052,96052,1574,0.513,0.037291,467,911,0.063222
3,91354,91354,32544,0.788,0.872146,15156,19234,0.010334
4,92026,92026,50321,0.765,0.744673,22870,29895,0.012158
...,...,...,...,...,...,...,...,...
1309,95448,95448,17407,0.794,0.893075,7988,10061,0.017021
1310,95901,95901,33455,0.632,0.166286,12571,19891,0.018237
1311,96088,96088,4690,0.577,0.085616,1466,2540,0.037082
1312,96118,96118,1779,0.663,0.248858,623,940,0.052888


In [83]:
# Change to float
employed_df['Employed Percentile'] = employed_df['Employed Percentile'].astype(float)
employed_df

,Zip Code,Location Name,Employed Population,Employed Value,Employed Percentile,Employed Numerator,Employed Denominator,se
0,92122,92122,48071,0.671,0.275495,23390,34859,0.016413
1,91384,91384,29212,0.552,0.058980,10803,19570,0.010942
2,96052,96052,1574,0.513,0.037291,467,911,0.063222
3,91354,91354,32544,0.788,0.872146,15156,19234,0.010334
4,92026,92026,50321,0.765,0.744673,22870,29895,0.012158
...,...,...,...,...,...,...,...,...
1309,95448,95448,17407,0.794,0.893075,7988,10061,0.017021
1310,95901,95901,33455,0.632,0.166286,12571,19891,0.018237
1311,96088,96088,4690,0.577,0.085616,1466,2540,0.037082
1312,96118,96118,1779,0.663,0.248858,623,940,0.052888


In [84]:
# check type
column_type = employed_df['Employed Percentile'].dtype
print(column_type)

float64


In [85]:
#income files
url = "https://api.healthyplacesindex.org/api/hpi?geography=zips&year=2015&indicator=percapitaincome&format=json&key=721a0a48-97f3-4a3f-a794-2bc855972617"
data = requests.get(url).json()
income_df = pd.DataFrame(data)
income_df

,geoid,name,population,value,percentile,numerator,denominator,se
0,92122,92122,48071,43620,0.690259,None,None,26516.717325
1,91384,91384,29212,33659,0.493912,None,None,20461.398176
2,96052,96052,1574,25815,0.287671,None,None,15693.009119
3,91354,91354,32544,49503,0.770928,None,None,30093.009119
4,92026,92026,50321,33163,0.474125,None,None,20159.878419
...,...,...,...,...,...,...,...,...
1309,95901,95901,33455,23771,0.227549,None,None,14450.455927
1310,96088,96088,4690,30335,0.410959,None,None,18440.729483
1311,96118,96118,1779,30580,0.417047,None,None,18589.665653
1312,96137,96137,3252,33337,0.484018,None,None,20265.653495


In [86]:
income_columns= {
    'geoid': 'Zip Code',
    'name': 'Location Name',
    'population': 'Income Population',
    'value': 'Income Value',
    'percentile': 'Income Percentile',
    'numerator': 'Income Numerator',
    'denominator': 'Income Denominator'
}

# Rename the columns using the dictionary
income_df.rename(columns=income_columns, inplace=True)

income_df

,Zip Code,Location Name,Income Population,Income Value,Income Percentile,Income Numerator,Income Denominator,se
0,92122,92122,48071,43620,0.690259,None,None,26516.717325
1,91384,91384,29212,33659,0.493912,None,None,20461.398176
2,96052,96052,1574,25815,0.287671,None,None,15693.009119
3,91354,91354,32544,49503,0.770928,None,None,30093.009119
4,92026,92026,50321,33163,0.474125,None,None,20159.878419
...,...,...,...,...,...,...,...,...
1309,95901,95901,33455,23771,0.227549,None,None,14450.455927
1310,96088,96088,4690,30335,0.410959,None,None,18440.729483
1311,96118,96118,1779,30580,0.417047,None,None,18589.665653
1312,96137,96137,3252,33337,0.484018,None,None,20265.653495


In [87]:
# Change to int
income_df['Income Value'] = income_df['Income Value'].astype(int)
income_df

,Zip Code,Location Name,Income Population,Income Value,Income Percentile,Income Numerator,Income Denominator,se
0,92122,92122,48071,43620,0.690259,None,None,26516.717325
1,91384,91384,29212,33659,0.493912,None,None,20461.398176
2,96052,96052,1574,25815,0.287671,None,None,15693.009119
3,91354,91354,32544,49503,0.770928,None,None,30093.009119
4,92026,92026,50321,33163,0.474125,None,None,20159.878419
...,...,...,...,...,...,...,...,...
1309,95901,95901,33455,23771,0.227549,None,None,14450.455927
1310,96088,96088,4690,30335,0.410959,None,None,18440.729483
1311,96118,96118,1779,30580,0.417047,None,None,18589.665653
1312,96137,96137,3252,33337,0.484018,None,None,20265.653495


In [88]:
# Merge employed_df and income_df on Zip Codes
merged_df = employed_df.merge(income_df, on="Zip Code", suffixes=("_employed", "_income"))

# Merge the resulting merged_df with hip_df on Zip Codes
hpi_poverty = merged_df.merge(hpi_df, on="Zip Code", suffixes=("_merged", "_hpi"))

In [89]:
# Change to float
hpi_poverty['Zip Code'] = hpi_poverty['Zip Code'].astype(int)
hpi_poverty

,Zip Code,Location Name_employed,Employed Population,Employed Value,Employed Percentile,Employed Numerator,Employed Denominator,se_employed,Location Name_income,Income Population,...,Income Numerator,Income Denominator,se_income,Location Name,Population,HPI Value,HPI Percentile,Numerator,Denominator,se
0,92122,92122,48071,0.671,0.275495,23390,34859,0.016413,92122,48071,...,None,None,26516.717325,92122,48071,0.192850,0.623762,None,None,None
1,91384,91384,29212,0.552,0.058980,10803,19570,0.010942,91384,29212,...,None,None,20461.398176,91384,29212,0.267728,0.666413,None,None,None
2,96052,96052,1574,0.513,0.037291,467,911,0.063222,96052,1574,...,None,None,15693.009119,96052,1574,-0.526667,0.168317,None,None,None
3,91354,91354,32544,0.788,0.872146,15156,19234,0.010334,91354,32544,...,None,None,30093.009119,91354,32544,0.511914,0.825590,None,None,None
4,92026,92026,50321,0.765,0.744673,22870,29895,0.012158,92026,50321,...,None,None,20159.878419,92026,50321,0.001588,0.503427,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1308,95448,95448,17407,0.794,0.893075,7988,10061,0.017021,95448,17407,...,None,None,34137.993921,95448,17407,0.558540,0.848439,None,None,None
1309,95901,95901,33455,0.632,0.166286,12571,19891,0.018237,95901,33455,...,None,None,14450.455927,95901,33455,-0.452014,0.201066,None,None,None
1310,96088,96088,4690,0.577,0.085616,1466,2540,0.037082,96088,4690,...,None,None,18440.729483,96088,4690,-0.257758,0.319117,None,None,None
1311,96118,96118,1779,0.663,0.248858,623,940,0.052888,96118,1779,...,None,None,18589.665653,96118,1779,-0.253218,0.326733,None,None,None


In [90]:
columns_to_drop = [
    "Location Name_employed",
    "Employed Numerator",
    "Employed Denominator",
    "Location Name_income",
    "Income Numerator",
    "Income Denominator",
    "Location Name",
    "Numerator",
    "Denominator",
    "Employed Value",
    "Employed Population",
    "Income Population",
    "Income Percentile",
    "Population",
    "HPI Percentile"
]

hpi_poverty.drop(columns=columns_to_drop, inplace=True)

In [91]:
vaccines_hpi_poverty_df = pd.merge(df, hpi_poverty, on='Zip Code', how='inner')
vaccines_hpi_poverty_df.head()

,Date,Zip Code,County Name,Total Population,Percent Fully Vaccinated,Percent Partially Vaccinated,Employed Percentile,se_employed,Income Value,se_income,HPI Value,se
0,2023-09-26,95460,Mendocino,2508.0,0.868022,0.072967,0.997717,0.040729,46675,28373.860182,0.125566,None
1,2023-09-26,95451,Lake,11603.0,0.593640,0.064811,0.213470,0.031611,33270,20224.924012,-0.160988,None
2,2023-09-26,92832,Orange,25837.0,0.719743,0.075783,0.482496,0.018237,28287,17195.744681,-0.154457,None
3,2023-09-26,95148,Santa Clara,48273.0,0.875810,0.056968,0.723364,0.009726,41013,24931.914894,0.419383,None
4,2023-09-26,94115,San Francisco,34604.0,0.858196,0.099468,0.934551,0.013374,89097,54162.310030,0.798741,None


In [92]:
vaccines_hpi_poverty_df["id"] = vaccines_hpi_poverty_df.index
vaccines_hpi_poverty_df.columns.values

array(['Date', 'Zip Code', 'County Name', 'Total Population',
       'Percent Fully Vaccinated', 'Percent Partially Vaccinated',
       'Employed Percentile', 'se_employed', 'Income Value', 'se_income',
       'HPI Value', 'se', 'id'], dtype=object)

In [93]:
colum_dp = [
    "Date",
    "Total Population",
    "Income Value",
]

vaccines_hpi_poverty_df.drop(columns=colum_dp, inplace=True)

In [94]:
vaccines_hpi_poverty_df.head()

,Zip Code,County Name,Percent Fully Vaccinated,Percent Partially Vaccinated,Employed Percentile,se_employed,se_income,HPI Value,se,id
0,95460,Mendocino,0.868022,0.072967,0.997717,0.040729,28373.860182,0.125566,None,0
1,95451,Lake,0.593640,0.064811,0.213470,0.031611,20224.924012,-0.160988,None,1
2,92832,Orange,0.719743,0.075783,0.482496,0.018237,17195.744681,-0.154457,None,2
3,95148,Santa Clara,0.875810,0.056968,0.723364,0.009726,24931.914894,0.419383,None,3
4,94115,San Francisco,0.858196,0.099468,0.934551,0.013374,54162.310030,0.798741,None,4


In [95]:
vaccines_hpi_poverty_names = {
    'Zip Code': 'zip_code',
    'Percent Fully Vaccinated': 'percent_fully_vaccinated',
    'Percent Partially Vaccinated': 'percent_partiall_vaccinated',
    'Employed Percentile': 'employed_percentile',
    'HPI Value': 'hpi_value',
    'County Name' : 'county'
}

# Rename the columns using the dictionary
vaccines_hpi_poverty_df.rename(columns=vaccines_hpi_poverty_names, inplace=True)

In [96]:
vaccines_hpi_poverty_df.head()

,zip_code,county,percent_fully_vaccinated,percent_partiall_vaccinated,employed_percentile,se_employed,se_income,hpi_value,se,id
0,95460,Mendocino,0.868022,0.072967,0.997717,0.040729,28373.860182,0.125566,None,0
1,95451,Lake,0.593640,0.064811,0.213470,0.031611,20224.924012,-0.160988,None,1
2,92832,Orange,0.719743,0.075783,0.482496,0.018237,17195.744681,-0.154457,None,2
3,95148,Santa Clara,0.875810,0.056968,0.723364,0.009726,24931.914894,0.419383,None,3
4,94115,San Francisco,0.858196,0.099468,0.934551,0.013374,54162.310030,0.798741,None,4


In [97]:
# colu_dp = [
#     "se_employed",
#     "se_income",
#     "se",
# ]

# vaccines_hpi_poverty_df.drop(columns=colu_dp, inplace=True)

In [98]:
vaccines_hpi_poverty_df.head()

,zip_code,county,percent_fully_vaccinated,percent_partiall_vaccinated,employed_percentile,se_employed,se_income,hpi_value,se,id
0,95460,Mendocino,0.868022,0.072967,0.997717,0.040729,28373.860182,0.125566,None,0
1,95451,Lake,0.593640,0.064811,0.213470,0.031611,20224.924012,-0.160988,None,1
2,92832,Orange,0.719743,0.075783,0.482496,0.018237,17195.744681,-0.154457,None,2
3,95148,Santa Clara,0.875810,0.056968,0.723364,0.009726,24931.914894,0.419383,None,3
4,94115,San Francisco,0.858196,0.099468,0.934551,0.013374,54162.310030,0.798741,None,4


## SQLite Creation and Population

In [99]:
# creates sqlite file
#conn = sq.connect('CA_COVID_data.sqlite'.format("case_surv")) 
# writes to file
#CA_case_sel_df.to_sql("case_surv", conn, if_exists='replace', index=False) 

In [100]:
CA_case_sel_df.head()

,id,county,year,month,age_group,sex,race,ethnicity,hosp_yn,icu_yn,death_yn,underlying_conditions_yn
0,0,ALAMEDA,2020,August,18 to 49 years,Male,NA,NA,No,Missing,Unknown,NaN
1,1,ALAMEDA,2020,August,65+ years,Female,Unknown,Unknown,Unknown,Missing,Unknown,NaN
2,2,ALAMEDA,2020,August,18 to 49 years,Male,NA,NA,No,Missing,Unknown,NaN
3,3,ALAMEDA,2020,August,18 to 49 years,Male,NA,NA,Unknown,Missing,Unknown,NaN
4,4,ALAMEDA,2020,August,18 to 49 years,Male,NA,NA,Unknown,Missing,Unknown,NaN


In [101]:
CA_case_sel_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 12 columns):
 #   Column                    Non-Null Count   Dtype 
---  ------                    --------------   ----- 
 0   id                        100000 non-null  int64 
 1   county                    100000 non-null  object
 2   year                      100000 non-null  int64 
 3   month                     100000 non-null  object
 4   age_group                 100000 non-null  object
 5   sex                       100000 non-null  object
 6   race                      100000 non-null  object
 7   ethnicity                 100000 non-null  object
 8   hosp_yn                   100000 non-null  object
 9   icu_yn                    100000 non-null  object
 10  death_yn                  100000 non-null  object
 11  underlying_conditions_yn  437 non-null     object
dtypes: int64(2), object(10)
memory usage: 9.2+ MB


In [102]:
#conn = sq.connect('CA_COVID_data.sqlite'.format("case_surv")) 
conn = sq.connect("../Data/CA_COVID_data.sqlite")
CA_case_sel_df.to_sql("case_surv", conn, if_exists='replace', index=False, dtype={'id': 'INT PRIMARY KEY'}) 

100000

In [103]:
engine = create_engine("sqlite:///../Data/CA_COVID_data.sqlite", echo=False)
inspector = inspect(engine)
inspector.get_table_names()

['case_surv']

In [104]:
pd.read_sql_table(table_name='case_surv', con=engine)

,id,county,year,month,age_group,sex,race,ethnicity,hosp_yn,icu_yn,death_yn,underlying_conditions_yn
0,0,ALAMEDA,2020,August,18 to 49 years,Male,NA,NA,No,Missing,Unknown,None
1,1,ALAMEDA,2020,August,65+ years,Female,Unknown,Unknown,Unknown,Missing,Unknown,None
2,2,ALAMEDA,2020,August,18 to 49 years,Male,NA,NA,No,Missing,Unknown,None
3,3,ALAMEDA,2020,August,18 to 49 years,Male,NA,NA,Unknown,Missing,Unknown,None
4,4,ALAMEDA,2020,August,18 to 49 years,Male,NA,NA,Unknown,Missing,Unknown,None
...,...,...,...,...,...,...,...,...,...,...,...,...
99995,99995,YUBA,2023,August,18 to 49 years,Female,White,Non-Hispanic/Latino,Unknown,Missing,Unknown,None
99996,99996,YUBA,2023,August,65+ years,Female,NA,NA,Unknown,Missing,Unknown,None
99997,99997,YUBA,2023,August,18 to 49 years,Female,White,Non-Hispanic/Latino,Unknown,Missing,Unknown,None
99998,99998,YUBA,2023,August,18 to 49 years,Female,White,Non-Hispanic/Latino,Unknown,Missing,Unknown,None


In [105]:
#conn = sq.connect('CA_COVID_data.sqlite'.format("vaccine_by_county")) # creates file
clean_complete_vaccine.to_sql("vaccine_by_county", conn, if_exists='replace', index=False, dtype={'id': 'INT PRIMARY KEY'})
# writes to file

2480

In [106]:
engine = create_engine("sqlite:///../Data/CA_COVID_data.sqlite", echo=False)
inspector = inspect(engine)
inspector.get_table_names()

['case_surv', 'vaccine_by_county']

In [107]:
pd.read_sql_table(table_name='vaccine_by_county', con=engine)

,county,year,month,cumulative_total_doses,cumulative_fully_vaccinated,cumulative_at_least_one_dose,cumulative_up_to_date_count,id,Date
0,ALAMEDA,2020,1,0,0,0,0,0,b'2020-01'
1,ALAMEDA,2020,7,1,0,1,0,1,b'2020-07'
2,ALAMEDA,2020,8,43,3,40,0,2,b'2020-07'
3,ALAMEDA,2020,9,104,45,59,0,3,b'2020-07'
4,ALAMEDA,2020,10,121,51,72,0,4,b'2020-07'
...,...,...,...,...,...,...,...,...,...
2475,YUBA,2023,5,109747,40271,44406,7025,2475,b'2023-10'
2476,YUBA,2023,6,109892,40278,44426,7154,2476,b'2023-10'
2477,YUBA,2023,7,109983,40279,44448,7239,2477,b'2023-10'
2478,YUBA,2023,8,110114,40280,44472,7364,2478,b'2023-10'


In [108]:
vaccines_hpi_poverty_df.to_sql("vaccines_hpi_poverty", 
                               conn, if_exists='replace', index=False, dtype={'id': 'INT PRIMARY KEY'})

30

In [109]:
inspector.get_table_names()

['case_surv', 'vaccine_by_county']

In [110]:
pd.read_sql_table(table_name='vaccines_hpi_poverty', con=engine)

,zip_code,county,percent_fully_vaccinated,percent_partiall_vaccinated,employed_percentile,se_employed,se_income,hpi_value,se,id
0,95460,Mendocino,0.868022,0.072967,0.997717,0.040729,28373.860182,0.125566,None,0
1,95451,Lake,0.593640,0.064811,0.213470,0.031611,20224.924012,-0.160988,None,1
2,92832,Orange,0.719743,0.075783,0.482496,0.018237,17195.744681,-0.154457,None,2
3,95148,Santa Clara,0.875810,0.056968,0.723364,0.009726,24931.914894,0.419383,None,3
4,94115,San Francisco,0.858196,0.099468,0.934551,0.013374,54162.310030,0.798741,None,4
5,92555,Riverside,0.624404,0.059705,0.522451,0.014590,16292.401216,-0.056020,None,5
6,92082,San Diego,0.721411,0.060305,0.381659,0.020061,23077.203647,0.078603,None,6
7,96134,Modoc,0.414764,0.058281,0.106164,0.046201,11634.650456,-0.488098,None,7
8,95311,Mariposa,0.480190,0.065504,0.098935,0.076596,17368.389058,-0.556753,None,8
9,91377,Ventura,0.835198,0.064029,0.646499,0.026748,38821.276596,0.710379,None,9


In [111]:
# conn.close() # good practice: close connection

In [112]:
conn = sq.connect('../Data/CA_COVID_data.sqlite')
cursor = conn.cursor()

In [113]:
with open('../Data/county_migration_data.geojson', 'r') as file:
    migration_gj = json.load(file)

In [114]:
# Create a new table to store the GeoJSON data
cursor.execute('''
CREATE TABLE IF NOT EXISTS us_county_migrations (          
    id INTEGER PRIMARY KEY,
    data TEXT        
)
''')

conn.commit()

In [115]:
# Serialize the dictionary
geojson_str = json.dumps(migration_gj)

# Insert the serialized string into the SQLite table
cursor.execute('''
INSERT INTO us_county_migrations (data) 
VALUES (?)
''', (geojson_str,))
conn.commit()
conn.close()

In [116]:
conn.close()